<a href="https://colab.research.google.com/github/Riju3323/JanataHack_HCare_Comp/blob/main/Copy_of_XGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialising all libraries.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Importing XGBM

In [ ]:
import xgboost as xgb

Accessing all provided data with Pandas

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/AV Proj/train.csv')
df_train_dict = pd.read_csv('/content/drive/MyDrive/AV Proj/train_data_dict.csv')
df_test = pd.read_csv('/content/drive/MyDrive/AV Proj/test.csv')
sample_op = pd.read_csv('/content/drive/MyDrive/AV Proj/sample_submission_lfbv3c3 (1).csv')

In [ ]:
df_train.head()
#printing train data set


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [ ]:
df_train.shape
#viewing train set shape.

(318438, 18)

In [ ]:
df_train.isnull().sum()
#checking for Null cells

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [ ]:
df_train.dtypes
#observing all feature types

case_id                                int64
Hospital_code                          int64
Hospital_type_code                    object
City_Code_Hospital                     int64
Hospital_region_code                  object
Available Extra Rooms in Hospital      int64
Department                            object
Ward_Type                             object
Ward_Facility_Code                    object
Bed Grade                            float64
patientid                              int64
City_Code_Patient                    float64
Type of Admission                     object
Severity of Illness                   object
Visitors with Patient                  int64
Age                                   object
Admission_Deposit                    float64
Stay                                  object
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
df_train['Stay'] = le.fit_transform(df_train['Stay'])
# Label encoding Stay feature and returning encoded values

In [ ]:
df_train['Stay'].value_counts()
#for viewing spread of encoded values

2     87491
1     78139
3     55159
5     35018
0     23604
4     11743
7     10254
10     6683
8      4838
9      2765
6      2744
Name: Stay, dtype: int64

Creating a merged test and train table.

In [ ]:
df_train['train_flag'] = 1
df_test['train_flag'] = 0
# Flags as separation marks between the two sets as we are about to merge.
df_test['Stay'] = None
#Creating this as a empty cell as test doesn't have it.
print(df_train.shape, df_test.shape)

df_a = pd.concat((df_train, df_test))
print(df_a.shape)

(318438, 19) (137057, 19)
(455495, 19)


In [ ]:
df_a.drop(['case_id','patientid'],axis=1,inplace=True)
# Removing this columns as low significance in evaluating stay duration.

In [ ]:
df_a['Bed Grade'] = np.where(df_a['Bed Grade'].isna(),1.0,df_a['Bed Grade'])
df_a['City_Code_Patient'] = np.where(df_a['City_Code_Patient'].isna(),0.0,df_a['City_Code_Patient'])
#As we saw these two features have null values, we are going to use pandas isna() to detect and replace them

df_a['Bed Grade'] = df_a['Bed Grade'].astype(int)
df_a['City_Code_Patient'] = df_a['City_Code_Patient'].astype(int)
df_a['Admission_Deposit'] = df_a['Admission_Deposit'].astype(int)
#As we saw, these three features have float types, so converting the float values into int values.

In [ ]:
df_a.dtypes
#observing all feature types again

Hospital_code                         int64
Hospital_type_code                   object
City_Code_Hospital                    int64
Hospital_region_code                 object
Available Extra Rooms in Hospital     int64
Department                           object
Ward_Type                            object
Ward_Facility_Code                   object
Bed Grade                             int64
City_Code_Patient                     int64
Type of Admission                    object
Severity of Illness                  object
Visitors with Patient                 int64
Age                                  object
Admission_Deposit                     int64
Stay                                 object
train_flag                            int64
dtype: object

In [ ]:
df_a.isnull().sum()
#all nulls removed except stay which we set as "None"

Hospital_code                             0
Hospital_type_code                        0
City_Code_Hospital                        0
Hospital_region_code                      0
Available Extra Rooms in Hospital         0
Department                                0
Ward_Type                                 0
Ward_Facility_Code                        0
Bed Grade                                 0
City_Code_Patient                         0
Type of Admission                         0
Severity of Illness                       0
Visitors with Patient                     0
Age                                       0
Admission_Deposit                         0
Stay                                 137057
train_flag                                0
dtype: int64

In [ ]:
temp=df_a.select_dtypes(['object']).columns
temp=temp[:-1]
temp
#removing stay column

Index(['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Age'],
      dtype='object')

In [ ]:
df_a = pd.get_dummies(data=df_a,columns=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Age'])
# creates dummy column for each unique value in this feature

In [ ]:
df_train, df_test = df_a[df_a.train_flag == 1], df_a[df_a.train_flag == 0]
df_train.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['Stay'], inplace=True, axis=1)
#Dropping these features before gbm
del df_a

In [ ]:
df_train.dtypes

Hospital_code                         int64
City_Code_Hospital                    int64
Available Extra Rooms in Hospital     int64
Bed Grade                             int64
City_Code_Patient                     int64
Visitors with Patient                 int64
Admission_Deposit                     int64
Stay                                 object
Hospital_type_code_a                  uint8
Hospital_type_code_b                  uint8
Hospital_type_code_c                  uint8
Hospital_type_code_d                  uint8
Hospital_type_code_e                  uint8
Hospital_type_code_f                  uint8
Hospital_type_code_g                  uint8
Hospital_region_code_X                uint8
Hospital_region_code_Y                uint8
Hospital_region_code_Z                uint8
Department_TB & Chest disease         uint8
Department_anesthesia                 uint8
Department_gynecology                 uint8
Department_radiotherapy               uint8
Department_surgery              

In [ ]:
df_train['Stay'] = df_train['Stay'].astype(int)
#setting Stay as int

In [ ]:
df_train, final = train_test_split(df_train, test_size=None, random_state=None, shuffle=True, stratify=df_train['Stay'])
#train size will automatically get alloted
#stratifying as Stay as it is dependent feature

In [ ]:
feature_cols = df_train.columns.tolist()
feature_cols.remove('Stay')
#feature_cols.remove('Admission_Deposit')
label_col = 'Stay'
print(feature_cols)

['Hospital_code', 'City_Code_Hospital', 'Available Extra Rooms in Hospital', 'Bed Grade', 'City_Code_Patient', 'Visitors with Patient', 'Admission_Deposit', 'Hospital_type_code_a', 'Hospital_type_code_b', 'Hospital_type_code_c', 'Hospital_type_code_d', 'Hospital_type_code_e', 'Hospital_type_code_f', 'Hospital_type_code_g', 'Hospital_region_code_X', 'Hospital_region_code_Y', 'Hospital_region_code_Z', 'Department_TB & Chest disease', 'Department_anesthesia', 'Department_gynecology', 'Department_radiotherapy', 'Department_surgery', 'Ward_Type_P', 'Ward_Type_Q', 'Ward_Type_R', 'Ward_Type_S', 'Ward_Type_T', 'Ward_Type_U', 'Ward_Facility_Code_A', 'Ward_Facility_Code_B', 'Ward_Facility_Code_C', 'Ward_Facility_Code_D', 'Ward_Facility_Code_E', 'Ward_Facility_Code_F', 'Type of Admission_Emergency', 'Type of Admission_Trauma', 'Type of Admission_Urgent', 'Severity of Illness_Extreme', 'Severity of Illness_Minor', 'Severity of Illness_Moderate', 'Age_0-10', 'Age_11-20', 'Age_21-30', 'Age_31-40', '

In [ ]:
cat_cols = feature_cols

In [ ]:
params = {}
params['learning_rate'] = 0.1
params['max_depth'] = 10
params['n_estimators'] = 100
params['min_child_weight'] = 1
params['subsample'] = 0.7
params['colsample_bytree']=0.7
params['scale_pos_weight']=1
params['reg_alpha'] = 1.6
params['reg_lambda'] = 1.2

In [ ]:
clf = xgb.XGBClassifier(**params)
    
clf.fit(df_train[feature_cols], df_train[label_col], early_stopping_rounds=100, eval_set=[(df_train[feature_cols], df_train[label_col]), (final[feature_cols], final[label_col])], eval_metric='mlogloss', verbose=True)

final_score = accuracy_score(final[label_col], clf.predict(final[feature_cols]))

print('Eval ACC: {}'.format(final_score))

[0]	validation_0-mlogloss:2.30364	validation_1-mlogloss:2.30707
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[1]	validation_0-mlogloss:2.21168	validation_1-mlogloss:2.21802
[2]	validation_0-mlogloss:2.13571	validation_1-mlogloss:2.14478
[3]	validation_0-mlogloss:2.07084	validation_1-mlogloss:2.0829
[4]	validation_0-mlogloss:2.0194	validation_1-mlogloss:2.03379
[5]	validation_0-mlogloss:1.97564	validation_1-mlogloss:1.99284
[6]	validation_0-mlogloss:1.93321	validation_1-mlogloss:1.9532
[7]	validation_0-mlogloss:1.8962	validation_1-mlogloss:1.91824
[8]	validation_0-mlogloss:1.8617	validation_1-mlogloss:1.88598
[9]	validation_0-mlogloss:1.83002	validation_1-mlogloss:1.85655
[10]	validation_0-mlogloss:1.80181	validation_1-mlogloss:1.83087
[11]	validation_0-mlogloss:1.77606	validation_1-mlogloss:1.80728
[12]	validation_0-mlogloss:1.75155	validation_1-mlogloss:1.78498
[1

In [ ]:
best_iter = clf.best_iteration
params['n_estimators'] = best_iter
print(params)

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 99, 'min_child_weight': 1, 'subsample': 0.7, 'colsample_bytree': 0.7, 'scale_pos_weight': 1, 'reg_alpha': 1.6, 'reg_lambda': 1.2}


In [ ]:
df_train = pd.concat((df_train, final))

In [ ]:
clf = xgb.XGBClassifier(**params)

clf.fit(df_train[feature_cols], df_train[label_col], eval_metric='merror', verbose=False)

eval_score_acc = accuracy_score(df_train[label_col], clf.predict(df_train[feature_cols]))

print('ACC: {}'.format(eval_score_acc))

ACC: 0.49255428058209133


In [ ]:
alg_1 = clf.predict(df_test[feature_cols])
alg_1 = le.inverse_transform(alg_1)
temp_1 = pd.read_csv('/content/drive/MyDrive/AV Proj/test.csv')
op = pd.DataFrame({'case_id': temp_1.case_id, 'Stay': alg_1})
op.to_csv('Av-Eivor-XGBM_2.csv', index=False)